In [93]:
from itertools import product
from random import shuffle
import copy
import numpy as np

In [160]:
class Scopa:
    
    def __init__(self):
        self.cards = [ (str(a+1) , b) for a, b in product(range(10), ('b', 'o', 's', 'c'))]
        self.deck = self._generate_deck()
        self.points_p1 = 0
        self.points_p2 = 0
        self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
        self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
        self.table_cards = [self.deck.pop(0) for _ in range(4)]
        self.turn = False
        
    def _generate_deck(self):
        deck = copy.copy(self.cards)
        shuffle(deck)
        return deck
    
    def _reset(self):
        self.cards = [ str(a+1) + b for a, b in product(range(10), ('b', 'o', 's', 'c'))]
        self.deck = self._generate_deck()
        self.points_p1 = 0
        self.points_p2 = 0
        self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
        self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
        self.table_cards = [self.deck.pop(0) for _ in range(4)]
        self.turn = False
        
    def _check_couple(self):
        #checks if there is a couple on the table. If there is,
        # it returns the couple without changing the table cards.
        #Double for loop: checks if the numbers are equal
        if not len(self.table_cards)>1: return False, '', ''
        for i in range(len(self.table_cards)):
            card1 = self.table_cards[i]
            n_card1 = int(card1[0])
            for j in range(len(self.table_cards)-i-1):
                k=i+j+1
                card2 = self.table_cards[k]
                n_card2 = int(card2[0])
                if n_card1 == n_card2:
                    return True, card1, card2
        return False, '', ''
                
    def move(self, card_n):
        point_scored_p1 = False
        point_scored_p2 = False
        if not self.turn:
            assert card_n < len(self.cards_p1)
            card_played = copy.copy(self.cards_p1[card_n])
            self.cards_p1.remove(card_played)
            self.table_cards.append(card_played)
            couple_on_table, couple1, couple2 = self._check_couple()
            if couple_on_table:
                self.points_p1+=1
                self.table_cards.remove(couple1)
                self.table_cards.remove(couple2)
                point_scored_p1 = True
            self.turn = not self.turn
            return [point_scored_p1, point_scored_p2, card_played]
            
        else:
            assert card_n < len(self.cards_p2)
            card_played = copy.copy(self.cards_p2[card_n])
            self.cards_p2.remove(card_played)
            self.table_cards.append(card_played)
            couple_on_table, couple1, couple2 = self._check_couple()
            if couple_on_table:
                self.points_p2+=1
                self.table_cards.remove(couple1)
                self.table_cards.remove(couple2)
            self.turn = not self.turn
            return [point_scored_p1, point_scored_p2, card_played]
            
    def play(self):
        self._reset()
        couple_table,_,_ = self._check_couple()
        while couple_table:
            self._reset()
            couple_table,_,_ = self._check_couple()
        print("There are still ", str(len(self.deck)), " cards.")
        while(len(self.deck)>0):
            print("Your cards :", self.cards_p1, " Table cards: ", self.table_cards)
            chosen_card = int(input("which card do you want to play?"))
            print("You played card: ", self.move(chosen_card-1)[2])
            print("New table after your move: ", self.table_cards)
            print("Your points :", self.points_p1)
            print("Your opponent played card: ", self.move(np.random.randint(0, len(self.cards_p2)))[2])
            print("New table after opponent move: ", self.table_cards)
            print("Opponent points :", self.points_p2)
            if len(self.cards_p1) == 0 and len(self.cards_p2) == 0:
                self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
                self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
                print("There are still ", str(len(self.deck)), " cards.")
        print("Game ended!")
        print("Your points :", self.points_p1)
        print("Opponent points :", self.points_p2)

In [260]:
class ScopaEnv:
    
    def __init__(self):
        self.cards = [ (str(a+1), b) for a, b in product(range(10), ('b', 'o', 's', 'c'))]
        self.deck = self._generate_deck()
        self.points_p1 = 0
        self.points_p2 = 0
        self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
        self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
        self.table_cards = [self.deck.pop(0) for _ in range(4)]
        self.turn = False
        
    def _generate_deck(self):
        deck = copy.copy(self.cards)
        shuffle(deck)
        return deck
    
    def feature_state(self, player):
        cards_list = [self.cards_p1, self.cards_p2]
        state = np.zeros((10 + 10)) #how many n-numbered card the player has and table has
        for card in cards_list[player]:
            n_card = int(card[0]) #n_card goes from 1 to 10
            state[n_card-1]+=1
        for card in self.table_cards:
            n_card = int(card[0]) #n_card goes from 1 to 10
            state[n_card-1+10]+=1
        return state.reshape(1,20)
            
    def _reset(self):
        self.cards = [ str(a+1) + b for a, b in product(range(10), ('b', 'o', 's', 'c'))]
        self.deck = self._generate_deck()
        self.points_p1 = 0
        self.points_p2 = 0
        self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
        self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
        self.table_cards = [self.deck.pop(0) for _ in range(4)]
        self.turn = False
        return self.feature_state, 0, False, ''
        
    def reset(self):
        return self._reset()
        

        
    def _check_couple(self):
        #checks if there is a couple on the table. If there is,
        # it returns the couple without changing the table cards.
        #Double for loop: checks if the numbers are equal
        if not len(self.table_cards)>1: return False, '', ''
        for i in range(len(self.table_cards)):
            card1 = self.table_cards[i]
            n_card1 = int(card1[0])
            for j in range(len(self.table_cards)-i-1):
                k=i+j+1
                card2 = self.table_cards[k]
                n_card2 = int(card2[0])
                if n_card1 == n_card2:
                    return True, card1, card2
        return False, '', ''
                
    def move(self, card_n):
        point_scored_p1 = False
        point_scored_p2 = False
        if not self.turn:
            assert card_n < len(self.cards_p1)
            card_played = copy.copy(self.cards_p1[card_n])
            self.cards_p1.remove(card_played)
            self.table_cards.append(card_played)
            couple_on_table, couple1, couple2 = self._check_couple()
            if couple_on_table:
                self.points_p1+=1
                self.table_cards.remove(couple1)
                self.table_cards.remove(couple2)
                point_scored_p1 = True
            self.turn = not self.turn
            return [point_scored_p1, point_scored_p2, card_played]
            
        else:
            assert card_n < len(self.cards_p2)
            card_played = copy.copy(self.cards_p2[card_n])
            self.cards_p2.remove(card_played)
            self.table_cards.append(card_played)
            couple_on_table, couple1, couple2 = self._check_couple()
            if couple_on_table:
                self.points_p2+=1
                self.table_cards.remove(couple1)
                self.table_cards.remove(couple2)
                point_scored_p2 = True
            self.turn = not self.turn
            return [point_scored_p1, point_scored_p2, card_played]
        
    def _draw_card(self):
        if len(self.cards_p1) == 0 and len(self.cards_p2) == 0:
            self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
            self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
        
    def action(self, card_n):
        reward = 0
        x = self.move(card_n)
        if x[0]: reward += 1
        #opponent move
        x = self.move(np.random.randint(0, len(self.cards_p2)))
        state = self.feature_state(0)
        done = len(self.deck) == 0 and len(self.cards_p1) == 0 and len(self.cards_p2) == 0
        if done: return state, reward, done, ''
        if len(self.cards_p1) == 0 and len(self.cards_p2) == 0:
            self._draw_card()
        
        return state, reward, done, ''
        
    def play(self):
        self._reset()
        couple_table,_,_ = self._check_couple()
        while couple_table:
            self._reset()
            couple_table,_,_ = self._check_couple()
        print("There are still ", str(len(self.deck)), " cards.")
        while(len(self.deck)>0):
            print("Your cards :", self.cards_p1, " Table cards: ", self.table_cards)
            chosen_card = int(input("which card do you want to play?"))
            print("You played card: ", self.move(chosen_card-1)[2])
            print("New table after your move: ", self.table_cards)
            print("Your points :", self.points_p1)
            print("Your opponent played card: ", self.move(np.random.randint(0, len(self.cards_p2)))[2])
            print("New table after opponent move: ", self.table_cards)
            print("Opponent points :", self.points_p2)
            if len(self.cards_p1) == 0 and len(self.cards_p2) == 0:
                self.cards_p1 = [self.deck.pop(0) for _ in range(3)]
                self.cards_p2 = [self.deck.pop(0) for _ in range(3)]
                print("There are still ", str(len(self.deck)), " cards.")
        print("Game ended!")
        print("Your points :", self.points_p1)
        print("Opponent points :", self.points_p2)

In [135]:
game = Scopa()

In [136]:
game.play()

There are still  30  cards.
Your cards : ['3o', '6b', '5o']  Table cards:  ['10b', '4s', '5s', '3c']
which card do you want to play?1
You played card:  3o
New table after your move:  ['10b', '4s', '5s']
Your points : 1
Your opponent played card:  9b
New table after opponent move:  ['10b', '4s', '5s', '9b']
Opponent points : 0
Your cards : ['6b', '5o']  Table cards:  ['10b', '4s', '5s', '9b']
which card do you want to play?1
You played card:  6b
New table after your move:  ['10b', '4s', '5s', '9b', '6b']
Your points : 1
Your opponent played card:  8b
New table after opponent move:  ['10b', '4s', '5s', '9b', '6b', '8b']
Opponent points : 0
Your cards : ['5o']  Table cards:  ['10b', '4s', '5s', '9b', '6b', '8b']
which card do you want to play?1
You played card:  5o
New table after your move:  ['10b', '4s', '9b', '6b', '8b']
Your points : 2
Your opponent played card:  3s
New table after opponent move:  ['10b', '4s', '9b', '6b', '8b', '3s']
Opponent points : 0
There are still  24  cards.
Yo

In [261]:
g = ScopaEnv()

In [284]:
print(g.cards_p1)
print(g.table_cards)
g.feature_state(0)

[('3', 'b'), ('5', 'o')]
[('1', 'b'), ('5', 'c')]


array([[ 0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [286]:
g.action(0)

(array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.]]), 1, True, '')

In [5]:
cards = [ str(a+1) + b for a, b in product(range(10), ('b', 'o', 's', 'c'))]

In [6]:
cards

['1b',
 '1o',
 '1s',
 '1c',
 '2b',
 '2o',
 '2s',
 '2c',
 '3b',
 '3o',
 '3s',
 '3c',
 '4b',
 '4o',
 '4s',
 '4c',
 '5b',
 '5o',
 '5s',
 '5c',
 '6b',
 '6o',
 '6s',
 '6c',
 '7b',
 '7o',
 '7s',
 '7c',
 '8b',
 '8o',
 '8s',
 '8c',
 '9b',
 '9o',
 '9s',
 '9c',
 '10b',
 '10o',
 '10s',
 '10c']